In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scikit.linear_model import LinearRegression
from scikit.model_selection import train_test_split
from scikit.metrics import mean_squared_error, mean_absolute_error, r2_score